In [2]:
from manim import *
from difflib import SequenceMatcher

[03/28/24 06:31:37] WARNING  Missing Plugins: {''}                                                   ]8;id=695023;file:///usr/local/lib/python3.11/site-packages/manim/plugins/__init__.py\__init__.py]8;;\:]8;id=604516;file:///usr/local/lib/python3.11/site-packages/manim/plugins/__init__.py#17\17]8;;\

In [3]:
def tokenize(code: Code) -> List[VMobjectFromSVGPath|Dot]:
    tokens = []
    for line in code[2]: # 0 = frame, 1 = dot, 2 = paragraph
        current_vmobject_group = []
        for vmobject in line:
            if isinstance(vmobject, Dot):
                if len(current_vmobject_group) > 0:
                    tokens.append(VGroup(*current_vmobject_group))
                    current_vmobject_group = []

                # tokens.append(vmobject)
                continue

            current_vmobject_group.append(vmobject)
            
        if len(current_vmobject_group) > 0:
            tokens.append(VGroup(*current_vmobject_group))

    return tokens

In [21]:
rust_code = "podman pull alpine"
token_replacements = "podman run -it --rm alpine sh"

output_code = """Trying to pull docker.io/library/alpine:latest...
Getting image source signatures
Copying blob 4abcf2066143 skipped: already exists  
Copying config 05455a0888 done   | 
Writing manifest to image destination
05455a08881ea9cf0e752bc48e61bbd71a34c029bb13df01e40e3e70e0d007bd"""
tert_output_code = "/ # "

In [25]:
%%manim -qm -v WARNING CodeBlock

class CodeBlock(Scene):
    def construct(self):
        code = Code(code=rust_code, language="rust", insert_line_no=False)
        code.shift(1.25 * UP + 4.6 * LEFT)

        replacement = Code(code=token_replacements, language="rust", insert_line_no=False)
        replacement.shift(1.25 * UP + 3.4 * LEFT)
        code = tokenize(code)
        replacement = tokenize(replacement)

        writes = [Write(token) for token in code]
        self.play(AnimationGroup(writes, lag_ratio=0.30))
        self.wait()

        secondary_code = Code(code=output_code, language='rust', insert_line_no=False)[2]
        self.play(Write(secondary_code))

        sequence_matcher = SequenceMatcher(None, rust_code.split(), token_replacements.split())
        opcodes = list(sequence_matcher.get_opcodes())

        unwrites = []
        for op, i0, i1, j0, j1 in opcodes:
            if op == 'delete' or op == 'replace':
                for x in range(i0, i1):
                    unwrites.append(Unwrite(code[x]))
        
        unwrites.append(FadeOut(secondary_code))
        self.play(AnimationGroup(*unwrites))

        transforms = []
        for op, i0, i1, j0, j1 in opcodes:
            if op == 'equal':
                for x, y in zip(range(i0, i1), range(j0, j1)):
                    transforms.append(Transform(code[x], replacement[y]))

        self.play(AnimationGroup(*transforms))

        writes = []
        for op, i0, i1, j0, j1 in opcodes:
            if op == 'insert' or op == 'replace':
                for x in range(j0, j1):
                    writes.append(Write(replacement[x]))

        self.play(AnimationGroup(*writes, lag_ratio=0.30))

        tertiary = Code(code=tert_output_code, language='rust', insert_line_no=False)[2]
        tertiary.shift(0.85 * UP + 6 * LEFT)
        self.play(Write(tertiary))
        self.wait()

Manim Community v0.18.0